<a href="https://colab.research.google.com/github/ayaanzhaque/SuiSense/blob/master/notebooks/BERT/workingBertModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.1MB 3.5MB/s 
     |████████████████████████████████| 890kB 16.1MB/s 
     |████████████████████████████████| 1.1MB 18.2MB/s 
     |████████████████████████████████| 3.0MB 27.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=2dc71c6b8c5ae2b4ae436ccf4795dde730e8d02a2fd32dc442f5eecd4e0c0c36
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
#importing relevant libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

import torch

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset
We'll use pandas to read the dataset and load it into a dataframe.

In [3]:
df = pd.read_csv('/content/real_data_for_model.csv')
og_batch_1 = df[['selftext', 'is_suicide']]

For performance reasons, we'll only use 2,000 sentences from the dataset

In [4]:
#batch_1_start = og_batch_1.head(60)
#batch_1_end = og_batch_1.tail(60)
#test_batch_1 = pd.concat([batch_1_start, batch_1_end], ignore_index=True)
batch_1 = og_batch_1.rename(columns={'selftext': 0, 'is_suicide': 1})
batch_1.head()

,0,1
0,I've been feeling really depressed and lonely ...,0
1,I literally broke down crying and asked to go ...,0
2,Any kind soul want to give a depressed person ...,0
3,literally though. have i done anything wrong,0
4,why does it hurt so much? Why can’t I be happy...,0


We can ask pandas how many sentences are labeled as "positive" (value 1) and how many are labeled "negative" (having the value 0)

In [5]:
batch_1[1].value_counts()

1    980
0    915
Name: 1, dtype: int64

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model. 

In [6]:
# For DistilBERT:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to so some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [7]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=128)))

In [8]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [9]:
np.array(padded).shape

(1895, 128)

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [10]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1895, 128)

In [11]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [12]:
features = last_hidden_states[0][:,0,:].numpy()
print(features)

[[ 0.19605026  0.20534156  0.24803329 ...  0.15781963  0.45455933
  -0.41783777]
 [-0.11692914 -0.39638266  0.09059574 ... -0.02261155  0.26243898
   0.33010307]
 [ 0.11361607 -0.04054473  0.23259798 ... -0.1438802   0.45649728
   0.47679883]
 ...
 [ 0.04628711  0.53464365 -0.19218408 ... -0.34011787  0.7708874
   0.6021303 ]
 [ 0.24700873  0.2810834   0.21017519 ... -0.1912898   0.2667738
  -0.13201511]
 [ 0.16641247  0.16862674  0.03572434 ... -0.15101409  0.5139446
   0.24030764]]


The labels indicating which sentence is positive and negative now go into the `labels` variable

In [13]:
labels = batch_1[1]
labels.head()

0    0
1    0
2    0
3    0
4    0
Name: 1, dtype: int64

In [19]:
np.savetxt("real_data_for_model_features.csv", features, delimiter=',')


In [20]:
features.shape

(1895, 768)

## Model #2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 2,000 sentences from the SST2 training set).

In [14]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.25, random_state=42, stratify=labels)

In [15]:
# parameters = {'C': np.linspace(0.0001, 100, 20)}
# grid_search = GridSearchCV(LogisticRegression(), parameters)
# grid_search.fit(train_features, train_labels)

# print('best parameters: ', grid_search.best_params_)
# print('best scrores: ', grid_search.best_score_)

In [16]:
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ModelCheckpoint

model3 = Sequential()
model3.add(Dense(1, activation='sigmoid'))

model3.add(Dense(10, activation='relu'))
# model3.add(Dense(1, activation='sigmoid'))
model3.add(Dense(3, activation = 'softmax')) # this gives the probability distribution for each class, should be 1 hot encoded, maybe we take this out of the model and only use it when predicting

model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


checkpoint = ModelCheckpoint('/content/depression_suicide_neither_nn.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  


history = model3.fit(train_features, train_labels, batch_size=32, epochs=100, verbose=1, callbacks=[checkpoint], validation_data=(test_features, test_labels))

Epoch 1/100
44/45 [============================>.] - ETA: 0s - loss: 0.7642 - accuracy: 0.1882
Epoch 00001: val_loss improved from inf to 0.76376, saving model to /content/depression_suicide_neither_nn.h5
45/45 [==============================] - 0s 6ms/step - loss: 0.7643 - accuracy: 0.1879 - val_loss: 0.7638 - val_accuracy: 0.0570
Epoch 2/100
 1/45 [..............................] - ETA: 0s - loss: 0.7737 - accuracy: 0.0312
Epoch 00002: val_loss improved from 0.76376 to 0.76374, saving model to /content/depression_suicide_neither_nn.h5
45/45 [==============================] - 0s 2ms/step - loss: 0.7640 - accuracy: 0.1485 - val_loss: 0.7637 - val_accuracy: 0.1857
Epoch 3/100
42/45 [===========================>..] - ETA: 0s - loss: 0.7660 - accuracy: 0.1637
Epoch 00003: val_loss improved from 0.76374 to 0.76374, saving model to /content/depression_suicide_neither_nn.h5
45/45 [==============================] - 0s 2ms/step - loss: 0.7640 - accuracy: 0.1640 - val_loss: 0.7637 - val_accurac

In [17]:
from keras.models import load_model

loadedModel = load_model('/content/depression_suicide_neither_nn.h5')
loadedModel.evaluate(test_features, test_labels)

15/15 [==============================] - 0s 1ms/step - loss: 0.7637 - accuracy: 0.8650


[0.7637370824813843, 0.8649789094924927]

<img src="https://jalammar.github.io/images/distilBERT/bert-training-logistic-regression.png" />

## Evaluating Model #2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [18]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.515 (+/- 0.04)


So our model clearly does better than a dummy classifier. But how does it compare against the best models?

## Proper SST2 scores
For reference, the [highest accuracy score](http://nlpprogress.com/english/sentiment_analysis.html) for this dataset is currently **96.8**. DistilBERT can be trained to improve its score on this task – a process called **fine-tuning** which updates BERT’s weights to make it achieve a better performance in this sentence classification task (which we can call the downstream task). The fine-tuned DistilBERT turns out to achieve an accuracy score of **90.7**. The full size BERT model achieves **94.9**.



And that’s it! That’s a good first contact with BERT. The next step would be to head over to the documentation and try your hand at [fine-tuning](https://huggingface.co/transformers/examples.html#glue). You can also go back and switch from distilBERT to BERT and see how that works.